In [1]:
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F

from datetime import datetime

In [2]:
spark = (
    SparkSession.builder
    .master('local[*]')
    .appName('test')
    .getOrCreate()
)

22/02/28 16:19:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
# Q1
spark.version

# '3.0.3'

'3.0.3'

In [32]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-27 01:13:08--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.21.131
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.21.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  28.2MB/s    in 24s     

2022-02-27 01:13:33 (29.1 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [3]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [4]:
df = (
    spark.read
    .option('header', 'true')
    .schema(schema)
    .csv('./fhvhv_tripdata_2021-02.csv')
)

In [34]:
df = df.repartition(24)

In [35]:
df.write.parquet('fhfhv/2021/02')

22/02/27 01:13:44 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 7, schema size: 8
CSV file: file:///home/igor/notebooks/fhvhv_tripdata_2021-02.csv


In [6]:
# Q2
!du -hs ./fhfhv/2021/02/

# 210MB

210M	./fhfhv/2021/02/


In [5]:
df = spark.read.parquet('fhfhv/2021/02')

In [13]:
# Q3
df.filter(F.to_date(df.pickup_datetime) == datetime(2021,2,15)).count()

# 367170

367170

In [31]:
# Q4
(
    df
    .withColumn('trip_duration', F.to_timestamp(df.dropoff_datetime) - F.to_timestamp(df.pickup_datetime))
    .withColumn('trip_duration_sec', F.to_timestamp(df.dropoff_datetime).cast('long') - F.to_timestamp(df.pickup_datetime).cast('long'))
    .orderBy(F.col('trip_duration_sec').desc())
    .select('hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'trip_duration_sec')
    .show()
)

# 2021-02-11

+-----------------+--------------------+-------------------+-------------------+-----------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|trip_duration_sec|
+-----------------+--------------------+-------------------+-------------------+-----------------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|            75540|
|           HV0004|              B02800|2021-02-17 15:54:53|2021-02-18 07:48:34|            57221|
|           HV0004|              B02800|2021-02-20 12:08:15|2021-02-21 00:22:14|            44039|
|           HV0003|              B02864|2021-02-03 20:24:25|2021-02-04 07:41:58|            40653|
|           HV0003|              B02887|2021-02-19 23:17:44|2021-02-20 09:44:01|            37577|
|           HV0003|              B02764|2021-02-25 17:13:35|2021-02-26 02:57:05|            35010|
|           HV0003|              B02875|2021-02-20 01:36:13|2021-02-20 11:16:19|            34806|
|         

In [38]:
# Q5
df.groupBy(F.col('dispatching_base_num')).count().orderBy(F.col('count').desc()).show()

# 2 stages

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [ ]:
df_zones = (
    spark.read
    .option('header', 'true')
    .csv('./taxi+_zone_lookup.csv')
)

In [11]:
df.registerTempTable('table_fhfhv')
df_zones.registerTempTable('table_zones')

In [15]:
# Q6
spark.sql('''
SELECT
    t2.Zone, t3.Zone, COUNT(*)
  FROM table_fhfhv t1
  LEFT JOIN table_zones t2
    ON t2.LocationID = t1.PULocationID
  LEFT JOIN table_zones t3
    ON t3.LocationID = t1.DOLocationID
 GROUP BY 1,2
 ORDER BY 3 DESC
 LIMIT 5
''').show()

# East New York East New York

+-------------------+-------------------+--------+
|               Zone|               Zone|count(1)|
+-------------------+-------------------+--------+
|      East New York|      East New York|   45041|
|       Borough Park|       Borough Park|   37329|
|           Canarsie|           Canarsie|   28026|
|Crown Heights North|Crown Heights North|   25976|
|          Bay Ridge|          Bay Ridge|   17934|
+-------------------+-------------------+--------+

